In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from NetEmbs import *

# Specify here the number of groups in your data. 
# I'm not sure should we use the finest strucre (25) or make preelemenatry merging... What do you think?
N_CLS = (14, 25)
# Min and Max number of clusters to investigate the chagnes in merging processes.
MIN_MAX_CLS = (8, 13)


IMAGE_FOLDER = "AlexFinal"
create_folder(IMAGE_FOLDER)


MODE = "SimulatedData"
DB_PATH = "Simulation/FSN_Data_5k.db"
# CONFIG.ROOT_FOLDER = "UvA/GoogleColab/Optimal 5k/"
CONFIG.ROOT_FOLDER = # TODO
CONFIG.EXPERIMENT = [0, 1]

#  ---------- CONFIG Setting HERE ------------
# .1 Sampling parameters
CONFIG.STRATEGY = "MetaDiff"
CONFIG.PRESSURE = 10
CONFIG.WINDOW_SIZE = 3
CONFIG.WALKS_PER_NODE = 50
CONFIG.WALKS_LENGTH = 10
# .2 TF parameters
CONFIG.STEPS = 50000
CONFIG.EMBD_SIZE = 8
CONFIG.LOSS_FUNCTION = "NegativeSampling"  # or "NCE"
CONFIG.BATCH_SIZE = 256
CONFIG.NEGATIVE_SAMPLES = 512

for strategy in ["MetaDiff", "OriginalRandomWalk", "DefUniform"]
    CONFIG.STRATEGY = strategy
    updateCONFIG(False)
    EMBS_PATH = CONFIG.WORK_FOLDER[0]+CONFIG.WORK_FOLDER[1]+CONFIG.WORK_FOLDER[2]

    if MODE == "SimulatedData":
        embs = pd.read_pickle(EMBS_PATH+"Embeddings.pkl")
        print("Embeddings have been uploaded to memory!")
        d = upload_data(DB_PATH, limit=None)
        d = prepare_data(d)
        print("Supported information has been uploaded to memory!")

    if MODE == "RealData":
        import extras
        import analysis

        embs = pd.read_pickle(EMBS_PATH+"Embeddings.pkl")

    print("Shape of uploaded Dataset is ", d.shape)
#     Explicitly say how manu clusters to be found: 14 and 25
    embs = cl_Agglomerative(embs, N_CLS[0])
    plot_tSNE(embs, legend_title="label", title=f"{IMAGE_FOLDER}/predicted14_{strategy}", context="talk_full")
    embs = cl_Agglomerative(embs, N_CLS[1])
    plot_tSNE(embs, legend_title="label", title=f"{IMAGE_FOLDER}/predicted25_{strategy}", context="talk_full")
    
# Return back to MetaDiff
CONFIG.STRATEGY = "MetaDiff"
updateCONFIG(False)
EMBS_PATH = CONFIG.WORK_FOLDER[0]+CONFIG.WORK_FOLDER[1]+CONFIG.WORK_FOLDER[2]
if MODE == "RealData":
    import extras
    import analysis
    embs = pd.read_pickle(EMBS_PATH+"Embeddings.pkl")

print("Shape of uploaded Dataset is ", d.shape)

for cl_n in range(7, 15):
    embs = cl_Agglomerative(embs, cl_n)
    embs.groupby(["GroundTruth", "label"])["ID"].count()\
        .unstack().fillna(0).apply(lambda x: x / x.sum(), axis = 1)\
        .to_excel(f"{IMAGE_FOLDER}/confusion{str(cl_n)}.xlsx")
    plot_tSNE(embs, legend_title="label", title=IMAGE_FOLDER+"/predicted_MetaDiff_"+str(cl_n), context="talk_half")
    print(f"Done with {cl_n} clusters!")